In [2]:
from pyspark.sql.functions import col, sum, count, avg

# =================================================================
# 0. CARGA DE DATOS (Asegurando que Spark vea los cambios de Silver)
# =================================================================
spark.catalog.refreshTable("lh_silver.fact_ventas")
df_fact = spark.read.table("lh_silver.fact_ventas")
df_dir = spark.read.table("lh_silver.dim_direccion")

# =================================================================
# 1. MÉTRICAS GLOBALES (Usando las columnas REALES de Silver)
# =================================================================
fact_global_negocio = df_fact.groupBy(
    "producto_id", 
    "fecha_id", 
    "codigo_postal_id",
    "proveedor_asignado", # <--- Columna real de Silver
    "estado_logistica"    # <--- Columna real de Silver
).agg(
    sum("monto_linea_total").alias("ingresos_totales"),
    sum("cantidad").alias("unidades_vendidas"),
    sum("gasto_logistico_total").alias("gasto_logistico_total"),
    avg("precio_venta_real").alias("precio_promedio_unidad"),
    count("orden_id").alias("total_pedidos")
)

# =================================================================
# 2. GUARDADO EN GOLD
# =================================================================
fact_global_negocio.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("lh_gold.fact_global_negocio")

df_dir.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("lh_gold.dim_direccion_gold")

print("ÉXITO: Capa Gold generada sin errores de columnas.")

StatementMeta(, da04ec19-5fcd-477e-87fb-9b14596ddbaa, 4, Finished, Available, Finished)

ÉXITO: Capa Gold generada sin errores de columnas.
